# ТИПИС 4 дз

Здесь буду впервые пытаться решить задачу классификации с горем и болью

Классификация - это тип задачи обучения с учителем, где целью является присвоение входным данным метки из заранее определённого набора категорий (классов)

**В этом датасете целевой переменной для задачи классификации будет переменная y — клиент подписал срочный депозит или нет.**

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [3]:
df = pd.read_csv('data/bank-full.csv', sep=";")

In [4]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


## Подготовка данных

**Выберите только указанные выше признаки.
Проверьте, есть ли пропущенные значения в этих признаках.**

In [5]:
# в задании требуется удалить столбцы default и loan
df = df.drop(['default', 'loan'], axis=1)
df.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no


In [6]:
# Проверка пропущенных значений
df.isnull().sum()

age          0
job          0
marital      0
education    0
balance      0
housing      0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

## Вопрос 1

**Какое самое частое значение (mode) для столбца education?**

In [7]:
df['education'].mode()

0    secondary
Name: education, dtype: object

## Вопрос 2

Создайте корреляционную матрицу для числовых признаков вашего датасета.

В корреляционной матрице вычислите коэффициент корреляции между каждой парой признаков.

**Какие два признака имеют наибольшую корреляцию?**

$$
\sigma = \sqrt{\frac{1}{n-1}\sum_{i=1}^{n}(x_i - \bar{x})^2}
$$
$$
r_{XY} = \frac{\sum_{i=1}^{n} (x_i - \bar{x})(y_i - \bar{y})}
{\sqrt{\sum_{i=1}^{n} (x_i - \bar{x})^2 \cdot \sum_{i=1}^{n} (y_i - \bar{y})^2}}
$$

In [8]:
corr_matrix = df[['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']].corr()
corr_matrix

,age,balance,day,duration,campaign,pdays,previous
age,1.000000,0.097783,-0.009120,-0.004648,0.004760,-0.023758,0.001288
balance,0.097783,1.000000,0.004503,0.021560,-0.014578,0.003435,0.016674
day,-0.009120,0.004503,1.000000,-0.030206,0.162490,-0.093044,-0.051710
duration,-0.004648,0.021560,-0.030206,1.000000,-0.084570,-0.001565,0.001203
campaign,0.004760,-0.014578,0.162490,-0.084570,1.000000,-0.088628,-0.032855
pdays,-0.023758,0.003435,-0.093044,-0.001565,-0.088628,1.000000,0.454820
previous,0.001288,0.016674,-0.051710,0.001203,-0.032855,0.454820,1.000000


## Кодирование целевой переменной

Теперь нам нужно закодировать переменную y.
Замените значения yes/no на 1/0.


In [9]:
# Создадим колонку target
# Такой способ придумал не сам, в курсике его взял

"""
df.loc[(df['y'] == 'no'), 'target'] = 0
df.loc[(df['y'] == 'yes'), 'target'] = 1
df = df.drop('y', axis=1)
df.head()
"""

# Чат ГПТ (вроде лучше)
df['target'] = df['y'].map({'no': 0, 'yes': 1}).astype(int)
df = df.drop('y', axis=1)
df.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,target
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,0


## Разделение данных

Разделите данные на **тренировочный/валидационный/тестовый** наборы с распределением 60%/20%/20%.

Используйте для этого Scikit-Learn (функцию train_test_split) и установите seed на 42.

Убедитесь, что целевая переменная y не включена в датафрейм.


In [10]:
# 1. разделяю на признаки и таргет, чуть шарю

X = df.drop('target', axis=1)
Y = df['target']

Вот дальше я вообще хз что я за магию делаю

In [11]:
# 2. делю на тренировочную и временную выборки (60/40)

X_train, X_temp, Y_train, Y_temp = train_test_split(
    X,
    Y,
    test_size=0.4,      # 40% уходит во временный набор
    random_state=42
)

In [12]:
# 3. делю временную выборку на валидационную и тестовую (по 20% каждая)
X_val, X_test, Y_val, Y_test = train_test_split(
    X_temp,
    Y_temp,
    test_size=0.5,      # 40% → 20% + 20%
    random_state=42
)

In [13]:
print(len(X_train), len(X_val), len(X_test))

27126 9042 9043


## Вопрос 3

Рассчитайте взаимную информацию (mutual_info_classif) между y и другими категориальными переменными в датасете.

Используйте только тренировочный набор.

Округлите результаты до двух знаков после запятой с помощью round(score, 2).

In [14]:
# посмотрим что вообще есть в contact, poutcome
df['contact'].value_counts()

contact
cellular     29285
unknown      13020
telephone     2906
Name: count, dtype: int64

In [15]:
df['poutcome'].value_counts()

poutcome
unknown    36959
failure     4901
other       1840
success     1511
Name: count, dtype: int64

короче скип этот вопрос, не понимаю как делать